In [1]:
import pandas as pd
import instance_generation

In [2]:
filepath = "C:/Users/Ellie\Documents/MASTERS/NLP/Project/parc_features/parc_features/parc_train_features.tsv"

In [3]:
df = pd.read_csv(filepath, delimiter='\t', index_col=0)

C:\Users\Ellie\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
instance_output = instance_generation.collect_instances_main(df)

In [9]:
print(instance_output[:4])

[[(58, 59), (57, 89), 0], [(60, 61), (57, 89), 0], [(65, 68), (57, 89), 0], [(72, 73), (57, 89), 0]]


In [15]:
gold_label_list = list()
pair_list = list()
for instance_list in instance_output:
    gold_label_list.append(instance_list[2])
    pair_tupel = (instance_list[0], instance_list[1])
    pair_list.append(pair_tupel)
    
print(gold_label_list[:4])
print(pair_list[:4])

[0, 0, 0, 0]
[((58, 59), (57, 89)), ((60, 61), (57, 89)), ((65, 68), (57, 89)), ((72, 73), (57, 89))]


In [16]:
def create_content_source_dist_feats(list_of_tuples, df):
    '''A function to add all content/source distance features to the df
    takes- list_of_tuples = a list of tuples of tuples of indices of the paired source/content spans
         - df = the dataframe containing the dataset
    returns- the df with added s/c_distance, num_conts_between and num_sources_between columns. Where X is used for tokens
    outside of the s/c pairs and numbers are given to all tokens inside of the s/c span to show the information in column'''
    
    df['s/c_distance'] = 'X'
    df['num_conts_between'] = 'X'
    df['num_sources_between'] = 'X'
    
    for instance in list_of_tuples:
        instance_list = list()
        index_list = list()
        
        source = instance[0]
        b1 = source[0]
        e1 = source[1]
        index_list.append(e1)
        for item in range(b1, e1):
            index_list.append(item)
        
        if e1 > b1:
            instance_list.append(e1)
        else:
            instance_list.append(b1)
        
        content = instance[1]
        b2 = content[0]
        e2 = content[1]
        index_list.append(e2)
        for item in range(b2, e2):
            index_list.append(item)
        
        for index in instance_list:
            if index < b2:
                if b2 < e2:
                    instance_list.append(b2)
                else:
                    instance_list.append(e2)
            elif index > b2:
                if b2 < e2:
                    instance_list.append(e2)
                else:
                    instance_list.append(b2)
        
        if instance_list[0] < instance_list[1]:
            distance = instance_list[1] - instance_list[0]
        else:
            distance = instance_list[0] - instance_list[1]
   
        df.loc[index_list,'s/c_distance'] = distance
    
        counter = 0
        for index in list(range(instance_list[0], instance_list[1])):
            if df.loc[index,'content_label_gold'] == 'B':
                counter += 1
                
        df.loc[index_list,'num_conts_between'] = counter
        
        counter = 0
        for index in list(range(instance_list[0], instance_list[1])):
            if df.loc[index,'candidate_source'] == 'B':
                counter += 1
                
        df.loc[index_list,'num_sources_between'] = counter
        
    return df

In [17]:
create_content_source_dist_feats(pair_list, df)

MemoryError: 